# Advanced Example: Random Environment Closed Queueing Network

This example demonstrates a closed queueing network that can be used in random environment models:
- **Nodes**: Delay node + PS Queue
- **Class**: Single closed class with N jobs
- **Service**: Parameterized exponential service rates

The model creates a simple 2-station closed network with circular routing.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun renvGenQN(rate: DoubleArray, N: Int): Network {
    val model = Network("qn1")
    
    // Block 1: nodes
    val delay = Delay(model, "Queue1")
    val queue = Queue(model, "Queue2", SchedStrategy.PS)
    
    // Block 2: classes
    val jobclass = ClosedClass(model, "Class1", N, delay, 0)
    
    // Set service rates
    delay.setService(jobclass, Exp(rate[0]))
    queue.setService(jobclass, Exp(rate[1]))
    
    // Block 3: topology (circular routing)
    val P = model.initRoutingMatrix()
    P.addRoute(jobclass, delay, queue, 1.0)
    P.addRoute(jobclass, queue, delay, 1.0)
    model.link(P)
    
    return model
}

// Create the model with example parameters
val model = renvGenQN(doubleArrayOf(2.0, 3.0), 5)

## About Random Environment Models

Random environment queueing models feature:
- **Base network**: A queueing network (like this closed QN)
- **Environment process**: A Markov chain that modulates network parameters
- **State-dependent rates**: Service/arrival rates depend on environment state

This simple closed network is a building block for:
- Markov-modulated queueing systems
- Systems with random breakdowns
- Networks with time-varying workloads

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")

// MVA Solver
val solverMVA = MVA(model)
val avgTableMVA = solverMVA.avgTable
println("\nMVA Solver:")
avgTableMVA.print()

// CTMC Solver
val solverCTMC = CTMC(model)
val avgTableCTMC = solverCTMC.avgTable
println("\nCTMC Solver:")
avgTableCTMC.print()

// Fluid Solver
val solverFluid = FLD(model)
val avgTableFluid = solverFluid.avgTable
println("\nFluid Solver:")
avgTableFluid.print()